In [2]:
# %% Derived from keras-rl
import opensim as osim
import numpy as np
import sys

import keras 
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, merge
from keras.optimizers import Adam

import numpy as np

from rl.agents import DDPGAgent
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess

sys.path = ["../"]+sys.path
from osim.env import *

from keras.optimizers import RMSprop

import argparse
import math

import matplotlib.pylab as plt


Using TensorFlow backend.


In [3]:
import opensim

#print(opensim.__path__)

import math
import numpy as np
import os
import random
from osim.env import OsimEnv

#import .osim
#print(OsimEnv.__path__)

class ArmEnv(OsimEnv):
    ninput = 14
    #model_path = os.path.join(os.path.dirname(__file__), './models/arm2dof6musc.osim')
    model_path = './models/arm2dof6musc.osim'
    #print(dir(OsimEnv))

    def __init__(self, visualize = False):
        self.iepisode = 0
        self.shoulder = 0.0
        self.elbow = 0.0
        super(ArmEnv, self).__init__(visualize = visualize)

    def configure(self):
        super(ArmEnv, self).configure()
        self.osim_model.joints.append(opensim.CustomJoint.safeDownCast(self.osim_model.jointSet.get(0)))
        self.osim_model.joints.append(opensim.CustomJoint.safeDownCast(self.osim_model.jointSet.get(1)))

    def new_target(self):
        #self.shoulder = -0.02275097#random.uniform(-1.2,0.3)
        #self.elbow = 0.2195816#random.uniform(-1.0,0)
        self.shoulder = 0.0#random.uniform(-1.2,0.3)
        self.elbow = 0.0#random.uniform(-1.0,0)
        #print(self.osim_model.joints[0].getCoordinate(0).getValue(self.osim_model.state))
        #print(self.osim_model.joints[0].getCoordinate(0).getValue(self.osim_model.state))
        #print(dir(self.osim_model.joints[0].set_coordinates))
        #print(self.osim_model.joints[0].set_coordinates.__doc__)
        #self.osim_model.joints[0].set_coordinates(0)
        #print(dir(self.osim_model.joints[0]))
        #self.osim_model.joints[1].set_coordinates(0,0.0)
        #action = [0,0,0,0,0,0]
        #muscle = self.osim_model.muscleSet.get(0)
        #muscle.setActivation(self.osim_model.state, float(action[0]))
        #muscleSet = self.osim_model.model.getMuscles()
        #muscle = muscleSet.get(0)
        #muscle.setActivation(self.osim_model.state, float(action[0]))
    

    def new_new(self):
        print(self.osim_model.joints[0].getCoordinate(0).getValue(self.osim_model.state))
        #self.osim_model.joints[0].getCoordinate(0).setValue(self.osim_model.state, -0.02275097)
        #self.osim_model.joints[1].getCoordinate(0).setValue(self.osim_model.state, 0.2195816)
        self.osim_model.joints[0].getCoordinate(0).setValue(self.osim_model.state, 0.0)
        self.osim_model.joints[1].getCoordinate(0).setValue(self.osim_model.state, 0.0)
        #muscle.setActivation(self.osim_model.state, float(action[0]))
        print(self.osim_model.joints[0].getCoordinate(0).getValue(self.osim_model.state))

    def reset(self):
        self.new_target()
        return super(ArmEnv, self).reset()

    def compute_reward(self):
        obs = self.get_observation()
        #pos = (self.angular_dist(obs[2],self.shoulder) + self.angular_dist(obs[3],self.elbow))
        #speed = 0 #(obs[4]**2 + obs[5]**2) / 200.0
        """
        if -0.1 < obs[2] and obs[2] < 0.1 and -0.1 < obs[3] and obs[3] < 0.1:
            return 1
        else:
            return 0
        #return - pos - speed
        #return 1.0
        """
        #costs = (obs[2]+0.3)**2 + (obs[3]-0.3)**2 + 0.1*(obs[4])**2 + (obs[5])**2
        costs = (obs[2]-0.4*np.cos(0.001*self.istep))**2 + (obs[3]-0.4*np.sin(0.001*self.istep))**2
        return -costs

    def get_observation(self):
        invars = np.array([0] * self.ninput, dtype='f')
        
        

        invars[0] = self.shoulder
        invars[1] = self.elbow
        
        invars[2] = self.osim_model.joints[0].getCoordinate(0).getValue(self.osim_model.state)
        invars[3] = self.osim_model.joints[1].getCoordinate(0).getValue(self.osim_model.state)

        invars[4] = self.osim_model.joints[0].getCoordinate(0).getSpeedValue(self.osim_model.state)
        invars[5] = self.osim_model.joints[1].getCoordinate(0).getSpeedValue(self.osim_model.state)

        invars[6] = self.sanitify(self.osim_model.joints[0].getCoordinate(0).getAccelerationValue(self.osim_model.state))
        invars[7] = self.sanitify(self.osim_model.joints[1].getCoordinate(0).getAccelerationValue(self.osim_model.state))

        pos = self.osim_model.model.calcMassCenterPosition(self.osim_model.state)
        vel = self.osim_model.model.calcMassCenterVelocity(self.osim_model.state)
        
        invars[8] = pos[0]
        invars[9] = pos[1]
        invars[10] = pos[2]

        invars[11] = vel[0]
        invars[12] = vel[1]
        invars[13] = vel[2]

        return invars

In [5]:
import opensim
import math
import numpy as np
import os
import random
from osim.env import OsimEnv

class ArmEnv(OsimEnv):
    ninput = 14
    model_path = './models/arm2dof6musc.osim'

    def __init__(self, visualize = False):
        self.iepisode = 0
        self.shoulder = 0.0
        self.elbow = 0.0
        super(ArmEnv, self).__init__(visualize = visualize)

    def configure(self):
        super(ArmEnv, self).configure()
        self.osim_model.joints.append(opensim.CustomJoint.safeDownCast(self.osim_model.jointSet.get(0)))
        self.osim_model.joints.append(opensim.CustomJoint.safeDownCast(self.osim_model.jointSet.get(1)))

    def new_target(self):
        self.shoulder = random.uniform(-1.2,0.3)
        self.elbow = random.uniform(-1.0,0)

    def reset(self):
        self.new_target()
        return super(ArmEnv, self).reset()

    def compute_reward(self):
        obs = self.get_observation()
        pos = (self.angular_dist(obs[2],self.shoulder) + self.angular_dist(obs[3],self.elbow))
        speed = 0 #(obs[4]**2 + obs[5]**2) / 200.0
        return - pos - speed

    def get_observation(self):
        invars = np.array([0] * self.ninput, dtype='f')

        invars[0] = self.shoulder
        invars[1] = self.elbow
        
        invars[2] = self.osim_model.joints[0].getCoordinate(0).getValue(self.osim_model.state)
        invars[3] = self.osim_model.joints[1].getCoordinate(0).getValue(self.osim_model.state)

        invars[4] = self.osim_model.joints[0].getCoordinate(0).getSpeedValue(self.osim_model.state)
        invars[5] = self.osim_model.joints[1].getCoordinate(0).getSpeedValue(self.osim_model.state)

        invars[6] = self.sanitify(self.osim_model.joints[0].getCoordinate(0).getAccelerationValue(self.osim_model.state))
        invars[7] = self.sanitify(self.osim_model.joints[1].getCoordinate(0).getAccelerationValue(self.osim_model.state))

        pos = self.osim_model.model.calcMassCenterPosition(self.osim_model.state)
        vel = self.osim_model.model.calcMassCenterVelocity(self.osim_model.state)
        
        invars[8] = pos[0]
        invars[9] = pos[1]
        invars[10] = pos[2]

        invars[11] = vel[0]
        invars[12] = vel[1]
        invars[13] = vel[2]

        return invars



In [4]:

#%% 
# Command line parameters
#parser = argparse.ArgumentParser(description='Train or test neural net motor controller')
#parser.add_argument('--train', dest='train', action='store_true', default=True)
#parser.add_argument('--test', dest='train', action='store_false', default=True)
#parser.add_argument('--steps', dest='steps', action='store', default=10000)
#parser.add_argument('--visualize', dest='visualize', action='store_true', default=False)
#parser.add_argument('--model', dest='model', action='store', default="example.h5f")
#args = parser.parse_args()


print("Arguments...")
#print(args)

args = argparse.Namespace(model='example_arm.h5f', steps=100000, visualize=False, train=True)
print(args)
# Load walking environment
print("Environment generated")

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('observation'))
        
 

Arguments...
Namespace(model='example_arm.h5f', steps=100000, train=True, visualize=False)
Environment generated


In [5]:
# Load walking environment
env = ArmEnv(args.visualize)


env.reset()
#env.new_new()
print(env.timestep_limit)
env.timestep_limit = 500

500


In [6]:
       
        


"""
for i in range(5000):
    observation, reward, done, info = env.step([0,0,0,0,0,0])#env.action_space.sample())
    #observation, reward, done, info = env.step(0.55*np.random.randint(0,2,6))#env.action_space.sample())
    #print(observation[2:4])
    print(observation[2])
"""
    
nb_actions = env.action_space.shape[0]

# Total number of steps in training
nallsteps = args.steps

# Next, we build a very simple model.
# Create networks for DDPG
actor = Sequential()
actor.add(Flatten(input_shape=(1,) + env.observation_space.shape))
actor.add(Dense(32))
actor.add(Activation('relu'))
actor.add(Dense(32))
actor.add(Activation('relu'))
actor.add(Dense(32))
actor.add(Activation('relu'))
actor.add(Dense(nb_actions))
actor.add(Activation('sigmoid'))
print(actor.summary())

action_input = Input(shape=(nb_actions,), name='action_input')
observation_input = Input(shape=(1,) + env.observation_space.shape, name='observation_input')
flattened_observation = Flatten()(observation_input)
x = merge([action_input, flattened_observation], mode='concat')
x = Dense(64)(x)
x = Activation('relu')(x)
x = Dense(64)(x)
x = Activation('relu')(x)
x = Dense(64)(x)
x = Activation('relu')(x)
x = Dense(1)(x)
x = Activation('linear')(x)
critic = Model(input=[action_input, observation_input], output=x)
print(critic.summary())

history = LossHistory()

# Set up the agent for training
memory = SequentialMemory(limit=100000, window_length=1)
random_process = OrnsteinUhlenbeckProcess(theta=.15, mu=0., sigma=.2, size=env.noutput)
agent = DDPGAgent(nb_actions=nb_actions, actor=actor, critic=critic, critic_action_input=action_input,
                  memory=memory, nb_steps_warmup_critic=100, nb_steps_warmup_actor=100,
                  random_process=random_process, gamma=.99, target_model_update=1e-3,
                  delta_clip=1.)
# agent = ContinuousDQNAgent(nb_actions=env.noutput, V_model=V_model, L_model=L_model, mu_model=mu_model,
#                            memory=memory, nb_steps_warmup=1000, random_process=random_process,
#                            gamma=.99, target_model_update=0.1)
agent.compile(Adam(lr=.001, clipnorm=1.), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
if args.train:
    agent.fit(env, nb_steps=nallsteps, visualize=False, verbose=2, nb_max_episode_steps=env.timestep_limit, log_interval=10000)
    
    #print history.losses
    # After training is done, we save the final weights.
    agent.save_weights(args.model, overwrite=True)

if not args.train:
    agent.load_weights(args.model)
    # Finally, evaluate our algorithm for 1 episode.
    agent.test(env, nb_episodes=1, visualize=False, nb_max_episode_steps=500, callbacks=[history])
    lst2 = [item[2] for item in history.losses]
    lst3 = [item[3] for item in history.losses]
    lst4 = [item[4] for item in history.losses]
    lst5 = [item[5] for item in history.losses]
    #print(lst2)
    plt.plot(lst2)
    #plt.plot(lst3)
    plt.plot(lst4)
    #plt.plot(lst5)
    #plt.plot(history.losses[0])
    plt.show()
    #print history.losses    
    #print(x)
    #for i in range(5000):
    #    observation, reward, done, info = env.step(env.action_space)#env.action_space.sample())
    #    print(observation)
    #agent.forward

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
flatten_1 (Flatten)              (None, 14)            0           flatten_input_1[0][0]            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 32)            480         flatten_1[0][0]                  
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 32)            0           dense_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 32)            1056        activation_1[0][0]               
___________________________________________________________________________________________

In [ ]:
import opensim as osim

print(osim.__version__)
print(keras.__version__)